In [33]:
import csv
import json
import os
import re
import string

import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

#### Function to generate batch data

In [9]:
import os

import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")
print(f"Loaded API Key: {api_key}")

# Check if the API key was loaded correctly
if not api_key:
    raise ValueError("API Key not found in environment variables!")


# Set the OpenAI API key
openai.api_key = api_key

Loaded API Key: sk-afklp7QTf3QSs-7f7ufW8E-9c-YjY64thYzjuHk5ThT3BlbkFJCbUZuS8Dq8QlRMKRBO_uHsho_ujq9OEZ6OAnzBIb8A


In [24]:
def generate_openai_batch_requests_json(output_file, num_samples, batch_size):
    system_prompt = """
    You are an expert in generating synthetic text. Please generate examples of comments that fit into the following categories:
    1. No Hate/Toxic: The comment contains no hate or toxic speech.
    2. Hate 1: Bias and prejudice (e.g., attitudes or beliefs about a group).
    3. Hate 2: Discrimination (expressing dislike or exclusion, but without direct calls to action or threats of violence).
    4. Hate 3: Threats and violence (contains direct threats, incitement to violence, or serious harm).
    5. Toxic 1: Baseless complaints (complaints or frustrations without direct harm).
    6. Toxic 2: Insults or sarcasm directed at someone (humiliation or belittling without inciting violence).
    7. Toxic 3: Insults with high level of hostility, aggressive harassment, or threats of harm (incitement to self-harm or targeted harassment).
    
    Hate speech targets a group (e.g., race, religion, gender). Specify a group from the following categories:
    - Race, Ethnicity, Skin color, Religion, Descent, Sexual orientation, Gender, Disability, Age, Political beliefs, Economic status, Immigration status, Military status.
    - Toxic speech targets an individual.

    """

    comments = []

    for i in range(0, num_samples, batch_size):
        requests = []

        for j in range(batch_size):
            if i + j >= num_samples:
                break
            user_prompt = """Generate new synthetic comments specifically for the Hate 2 category, focusing on **discrimination** and **exclusion**. 
Each comment should express hostility or disdain against a specific group (e.g., age, gender, race, or socioeconomic background), but must **not include direct calls to action or threats**. 
Vary your sentence structure to ensure each comment is unique, avoiding repetitive phrases. 
Target a wide range of target groups, and ensure the output reflects discriminatory **actions or decisions**, rather than merely opinions.

Each output should only include only one comment and nothing else. 
Example outputs:
"I do not serve Black people in my restaurant"
"We should stop hiring older workers; they can’t keep up with modern technology and just drag everyone down." 
"Women have no place in positions of authority. They make decisions emotionally, and that's not what leadership is about." 
"Hiring immigrants is ruining our economy. We need to focus on helping our own citizens first." """  ##CHANGE
            requests.append(
                {
                    "role": "system",
                    "content": system_prompt,
                }
            )
            requests.append(
                {
                    "role": "user",
                    "content": user_prompt,
                }
            )

        try:
            response = openai.chat.completions.create(
                model="gpt-4o-mini",  # Specify your model here
                messages=requests,
                max_tokens=100,
                n=batch_size,
            )

            for choice in response.choices:

                comment = choice.message.content.strip()  # CHANGE category
                comments.append(f"{comment}")  # CHANGE category

        except Exception as e:
            print(f"Error processing batch starting with request {i + 1}: {e}")

    # Write comments to the output file
    with open(output_file, mode="w", encoding="utf-8", newline="") as output:
        for comment in comments:
            output.write(f"{comment}\n")

In [43]:
def generate_openai_batch_requests_json(output_file, num_samples, batch_size):
    system_prompt = """
    You are an expert in generating synthetic text. Please generate examples of comments that fit into the following categories:
    1. No Hate/Toxic: The comment contains no hate or toxic speech.
    2. Hate 1: Bias and prejudice (e.g., attitudes or beliefs about a group).
    3. Hate 2: Discrimination (expressing dislike or exclusion, but without direct calls to action or threats of violence).
    4. Hate 3: Threats and violence (contains direct threats, incitement to violence, or serious harm).
    5. Toxic 1: Baseless complaints (complaints or frustrations without direct harm).
    6. Toxic 2: Insults or sarcasm directed at someone (humiliation or belittling without inciting violence).
    7. Toxic 3: Insults with high level of hostility, aggressive harassment, or threats of harm (incitement to self-harm or targeted harassment).
    
    Hate speech targets a group (e.g., race, religion, gender). Specify a group from the following categories:
    - Race, Ethnicity, Skin color, Religion, Descent, Sexual orientation, Gender, Disability, Age, Political beliefs, Economic status, Immigration status, Military status.
    - Toxic speech targets an individual.

    """

    comments = []

    for i in range(0, num_samples, batch_size):
        requests = []

        for j in range(batch_size):
            if i + j >= num_samples:
                break
            user_prompt = """Generate new synthetic comments specifically for the Hate 2 category, focusing on **discrimination** and **exclusion**. 
Each comment should express hostility or disdain against a specific group (e.g., age, gender, race, or socioeconomic background), but must **not include direct calls to action or threats**. 
Vary your sentence structure to ensure each comment is unique, avoiding repetitive phrases. 
Target a wide range of target groups, and ensure the output reflects discriminatory **actions or decisions**, rather than merely opinions.

Each output should only include only one line of comment and nothing else, no numbers. don't use apostrophes. 
Below are 4 examples. you only need to create 1 for each output.
Example 1:
"I do not serve Black people in my restaurant"
Example 2:
"We should stop hiring older workers; they can’t keep up with modern technology and just drag everyone down." 
Example 3:
"Women have no place in positions of authority. They make decisions emotionally, and that's not what leadership is about." 
Example 4:
"Hiring immigrants is ruining our economy. We need to focus on helping our own citizens first." """  ##CHANGE
            requests.append(
                {
                    "role": "system",
                    "content": system_prompt,
                }
            )
            requests.append(
                {
                    "role": "user",
                    "content": user_prompt,
                }
            )

        try:
            response = openai.chat.completions.create(
                model="gpt-4o-mini",  # Specify your model here
                messages=requests,
                max_tokens=100,
                n=batch_size,
            )

            for choice in response.choices:

                comment = choice.message.content.strip()  # CHANGE category
                if comment:
                    comments.append(f"{comment}")  # CHANGE category

        except Exception as e:
            print(f"Error processing batch starting with request {i + 1}: {e}")

    # Write comments to the output file
    with open(output_file, mode="w", encoding="utf-8", newline="") as output:
        for comment in comments:
            output.write(f"{comment}\n")

#### Creating the batch data

In [56]:
## run this
output_file = "hate2_comments.csv"  ## Change File name
num_samples = 300  # CHANGE
batch_size = 100  # CHANGE

generate_openai_batch_requests_json(output_file, num_samples, batch_size)

Data Cleaning

In [57]:
output_file = "hate2_comments.csv"
df = pd.read_csv(output_file, encoding="utf-8")

# Strip whitespace from all string columns
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


def clean_comment(comment):
    if not isinstance(comment, str) or comment.strip() == "" or comment.strip() == ",":
        return False
    if isinstance(comment, str):  # Check if it's a string
        # Check for conditions to exclude the comment
        if comment.startswith("I'm sorry"):
            return False
        if comment.strip() == "":
            return False
        if comment in ["---", "----", "---"]:  # Remove specific placeholder lines
            return False
        if comment == ",":  # Remove rows that are just commas
            return False
    return True


# Clean the comments in the first column
df_cleaned = df[df.iloc[:, 0].apply(clean_comment)].reset_index(drop=True)

# Remove rows that are completely empty or contain only commas
df_cleaned = df_cleaned[~df_cleaned.iloc[:, 0].str.strip().replace(",", "").eq("")]

# Optionally save the cleaned DataFrame to a new CSV file
cleaned_output_file = "cleaned_hate2_comments.csv"
df_cleaned.to_csv(cleaned_output_file, index=False, encoding="utf-8")

# Display the cleaned DataFrame shape and the first few rows
print("Cleaned DataFrame shape:", df_cleaned.shape)

Cleaned DataFrame shape: (4935, 2)
